In [1]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2

  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.4
    Uninstalling numpy-1.19.4:
      Successfully uninstalled numpy-1.19.4
  Rolling back uninstall of numpy
  Moving to c:\users\jordan\appdata\roaming\python\python38\scripts\f2py.exe
   from C:\Users\jordan\AppData\Local\Temp\pip-uninstall-l0u6wfyo\f2py.exe
  Moving to c:\users\jordan\appdata\roaming\python\python38\site-packages\numpy-1.19.4.dist-info\
   from C:\Users\jordan\AppData\Roaming\Python\Python38\site-packages\~umpy-1.19.4.dist-info
  Moving to c:\users\jordan\appdata\roaming\python\python38\site-packages\numpy\
   from C:\Users\jordan\AppData\Roaming\Python\Python38\site-packages\~umpy


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\numpy\\LICENSE.txt'
Consider using the `--user` option or check the permissions.



  Using cached tensorflow_estimator-2.4.0-py2.py3-none-any.whl (462 kB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0


In [1]:
import gym
import random

In [2]:
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [3]:
episodes = 50
Q = []
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = random.choice([0,1])
        n_state, reward, done, info = env.step(action)
        score += reward
    print("Episode:{} Score:{}".format(episode, score))
    Q.append(score)

Episode:1 Score:17.0
Episode:2 Score:14.0
Episode:3 Score:33.0
Episode:4 Score:22.0
Episode:5 Score:17.0
Episode:6 Score:33.0
Episode:7 Score:49.0
Episode:8 Score:20.0
Episode:9 Score:18.0
Episode:10 Score:18.0
Episode:11 Score:21.0
Episode:12 Score:41.0
Episode:13 Score:44.0
Episode:14 Score:26.0
Episode:15 Score:13.0
Episode:16 Score:36.0
Episode:17 Score:17.0
Episode:18 Score:15.0
Episode:19 Score:10.0
Episode:20 Score:25.0
Episode:21 Score:21.0
Episode:22 Score:18.0
Episode:23 Score:22.0
Episode:24 Score:44.0
Episode:25 Score:21.0
Episode:26 Score:13.0
Episode:27 Score:19.0
Episode:28 Score:15.0
Episode:29 Score:21.0
Episode:30 Score:25.0
Episode:31 Score:15.0
Episode:32 Score:16.0
Episode:33 Score:14.0
Episode:34 Score:14.0
Episode:35 Score:23.0
Episode:36 Score:33.0
Episode:37 Score:14.0
Episode:38 Score:27.0
Episode:39 Score:15.0
Episode:40 Score:13.0
Episode:41 Score:17.0
Episode:42 Score:36.0
Episode:43 Score:21.0
Episode:44 Score:13.0
Episode:45 Score:28.0
Episode:46 Score:26

In [5]:
print(Q)

[17.0, 14.0, 33.0, 22.0, 17.0, 33.0, 49.0, 20.0, 18.0, 18.0, 21.0, 41.0, 44.0, 26.0, 13.0, 36.0, 17.0, 15.0, 10.0, 25.0, 21.0, 18.0, 22.0, 44.0, 21.0, 13.0, 19.0, 15.0, 21.0, 25.0, 15.0, 16.0, 14.0, 14.0, 23.0, 33.0, 14.0, 27.0, 15.0, 13.0, 17.0, 36.0, 21.0, 13.0, 28.0, 26.0, 14.0, 32.0, 11.0, 10.0]


In [7]:
print( (sum(Q)) / (len(Q) ))

22.0


In [12]:
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [13]:
def build_model(states, actions):
    model = tensorflow.keras.Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [14]:
model = build_model(states, actions)

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 24)                120       
_________________________________________________________________
dense_4 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


In [16]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [17]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [18]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)


C:\Users\jordan\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


    1/10000 [..............................] - ETA: 44:39 - reward: 1.0000

C:\ProgramData\Anaconda3\lib\site-packages\rl\memory.py:40: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 77s 8ms/step - reward: 1.0000
101 episodes - episode_reward: 98.752 [10.000, 200.000] - loss: 2.022 - mae: 18.215 - mean_q: 36.863

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 77s 8ms/step - reward: 1.0000
52 episodes - episode_reward: 191.827 [165.000, 200.000] - loss: 3.276 - mae: 35.522 - mean_q: 71.753

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 77s 8ms/step - reward: 1.0000
50 episodes - episode_reward: 197.320 [160.000, 200.000] - loss: 3.503 - mae: 38.691 - mean_q: 78.015

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 79s 8ms/step - reward: 1.0000
51 episodes - episode_reward: 196.824 [145.000, 200.000] - loss: 4.677 - mae: 39.628 - mean_q: 79.740

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 79s 8ms/step - reward: 1.0000
done, took 388.494 seconds


In [19]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 131.000, steps: 131
Episode 2: reward: 149.000, steps: 149
Episode 3: reward: 167.000, steps: 167
Episode 4: reward: 169.000, steps: 169
Episode 5: reward: 140.000, steps: 140
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 159.000, steps: 159
Episode 8: reward: 131.000, steps: 131
Episode 9: reward: 175.000, steps: 175
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 155.000, steps: 155
Episode 13: reward: 144.000, steps: 144
Episode 14: reward: 153.000, steps: 153
Episode 15: reward: 133.000, steps: 133
Episode 16: reward: 133.000, steps: 133
Episode 17: reward: 192.000, steps: 192
Episode 18: reward: 158.000, steps: 158
Episode 19: reward: 133.000, steps: 133
Episode 20: reward: 200.000, steps: 200
Episode 21: reward: 116.000, steps: 116
Episode 22: reward: 128.000, steps: 128
Episode 23: reward: 200.000, steps: 200
Episode 24: reward: 144.000, steps: 144
Episode 25: reward: 

In [20]:
_ = dqn.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...
Episode 1: reward: 149.000, steps: 149
Episode 2: reward: 191.000, steps: 191
Episode 3: reward: 177.000, steps: 177
Episode 4: reward: 132.000, steps: 132
Episode 5: reward: 190.000, steps: 190
